# C7082 Assignment: Mushroom classification
Student number: 23381200
Date: xx-xx-xx

# Convolutional Neural Network Training through Transfer Learning
Transfer learning using InceptionV3

In [ ]:
# Load necessary libraries
import os 

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random

import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dropout, Dense

!pip install torchvision
from torchvision import transforms



In [ ]:
# Dataset: insert URL


path_to_dataset = "mushroom_dataset"
train_path = path_to_dataset +"/train"
test_path = path_to_dataset + "/test"
valid_path = path_to_dataset + "/valid"


no_of_classes = len(os.listdir(train_path))
print("No. of Classes: " + str(no_of_classes))



In [ ]:


# NOT CURRENTLY WORKING - EXCLUDE FOR NOW
# view a random image
def view_random_image(target_dir, target_class) :
    target_folder = target_dir + "/" + target_class
    # get the random image
    random_image = random.sample(os.listdir(target_folder), 1)
    # show the image
    img = mping.imread(target_folder + "/" + random_image[0])
    plt.imshow(img)
    plt.title(target_class)
    plt.axis("off")
    plt.show()

    print(f"Image shape: {img.shape}")
    return img

img = view_random_image(target_dir=train_path, target_class='poisonous')

In [ ]:
# Build the model

In [ ]:

# Image preprocessing

# Data augmentation to generate more data
# Define your augmentation parameters
datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.inception_v3.preprocess_input,  # Your preprocessing function
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

def custom_preprocessing(img):
    """
    Applies InceptionV3 preprocessing, Color Jitter, and standardization.
    """
    img = tf.keras.applications.inception_v3.preprocess_input(img)
    img = color_jitter(img)
    img = (img - img.mean()) / img.std()  # Standardization
    return img

# Create batches with augmentation for training data
train_batches = datagen.flow_from_directory(
    directory=train_path, 
    target_size=(299,299), 
    classes=['edible', 'poisonous'], 
    batch_size=32)


# For validation and test data, only apply preprocessing (no augmentation)
valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.inception_v3.preprocess_input) \
    .flow_from_directory(directory=valid_path, 
                         target_size=(299,299), 
                         classes=['edible', 'poisonous'], 
                         batch_size=32)


test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.inception_v3.preprocess_input) \
    .flow_from_directory(directory=test_path, 
                         target_size=(299,299), 
                         classes=['edible', 'poisonous'], 
                         batch_size=32, 
                         shuffle=False)


In [4]:
# Visualise the data
imgs, labels = next(train_batches)

# Plotting function to plot the processed images
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 10, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
plotImages(imgs)
print(labels)

In [6]:
# create the model
base_model = tf.keras.applications.InceptionV3(include_top=False,  # exclude the top since we are training the model for a new task
    weights="imagenet",
    input_tensor=None,
    input_shape=(299,299,3),                       # there is only edible or poisonous
    classifier_activation="softmax",
    name="incV3_base",
)

# freeze the model
base_model.trainable = False

In [7]:
# Create the inputs into the model
inputs = tf.keras.layers.Input(shape=(299,299,3), name = "input_layer")

In [ ]:
# Pass the inputs 
x = base_model(inputs)
print(f"The model shape after passing the inputs: {x.shape}")


In [ ]:

# Average pool layer the outputs of the base model
x = tf.keras.layers.GlobalAveragePooling2D(name = "Global-average-pooling-layer")(x)
print(f"The shape after GlobalAveragePoolid2D: {x.shape}")


In [10]:
# Add a Dense layer with trainable parameters
x = Dense(units=32, activation='relu')(x) 

In [11]:
# Add Dropout layer
x = Dropout(rate=0.5)(x)  # Adjust dropout rate as needed

In [12]:
# Add a Dense layer with trainable parameters
x = Dense(units=16, activation='relu')(x) 

In [13]:
# Add Dropout layer
x = Dropout(rate=0.4)(x)  # Adjust dropout rate as needed

In [14]:

# Create the last output layer
outputs = tf.keras.layers.Dense(no_of_classes, activation='softmax', name='output-layer')(x)


In [15]:

# Merge the inputs and outputs into one model
model = tf.keras.Model(inputs, outputs)


In [15]:
# Lets play with training the model
# Set all layers to non-trainable initially
for layer in model.layers:
    layer.trainable = False

# Make the last 2 layers trainable
for layer in model.layers[-3:]:
    layer.trainable = True

In [ ]:

model.compile(loss = "categorical_crossentropy",
              optimizer= tf.keras.optimizers.Adam(learning_rate=0.01),
              metrics = ["accuracy"])

print(model.summary())

In [17]:
EPOCHS = 10

In [18]:
best_model_file = "best_model.keras"

from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

callbacks = [
    ModelCheckpoint(best_model_file, verbose=1, save_best_only=True, monitor="val_accuracy"),
    ReduceLROnPlateau(monitor="val_accuracy", patience=4, factor=0.1, verbose=1, min_lr=1e-6),
    EarlyStopping(monitor="val_accuracy", patience=5, verbose=1)
]

In [ ]:
# Train the model

history = model.fit(train_batches,
                    epochs=EPOCHS,
                    steps_per_epoch= len(train_batches),
                    validation_data=valid_batches,
                    validation_steps=len(valid_batches),
                    callbacks=callbacks)

In [ ]:
# evaluate the test data

print(model.evaluate(test_batches))


In [ ]:
print(len(history.history["loss"]))
print(len(history.history["val_loss"]))
print(len(history.history["accuracy"]))
print(len(history.history["val_accuracy"]))

In [ ]:

# plot the results

def plot_loss_curves(history):
    loss = history.history["loss"]
    val_loss = history.history["val_loss"]
    accuracy = history.history["accuracy"]
    val_accuracy = history.history["val_accuracy"]

    epochs = range(len(loss))

    # plot the loss
    plt.plot(epochs, loss, label = "training loss")
    plt.plot(epochs, val_loss, label="val_loss")
    plt.title("Loss")
    plt.xlabel("Epochs")
    plt.legend()
    plt.show()

      # plot the accuracy
    plt.plot(epochs, accuracy, label = "training accuracy")
    plt.plot(epochs, val_accuracy, label="validation accuracy")
    plt.title("Accuracy")
    plt.xlabel("Epochs")
    plt.legend()
    plt.show()

# run the function
plot_loss_curves(history)


